<a href="https://colab.research.google.com/github/jonathantcallahan/guidance/blob/main/book_processing_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install openai
%pip install getpass
%pip install pypdf

In [2]:
import openai
from openai import OpenAI
import os

client = OpenAI()

In [3]:
training_json = []

In [17]:
print(len('asdf'))

4


In [53]:
import uuid

def store_json(chunks, targ):
    for chunk in chunks:
        if len(chunk) < 30:
            continue
        json_obj = { 
            "id" : str(uuid.uuid4()), 
            "instruction" : "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.", 
            "input" : "", 
            "output" : chunk 
        }
        targ.append(json_obj)

In [54]:
def chunk_gpt(text, targ):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are a document processor used to create fine-tuning data. You will receive 5000 characters worth of a book, and extract portions of text that would be coherent as the answer to a theoretical, unspecified question. Each answer can be up to 200 words. The cohesive answers within the text may directly following each other and there may be space between them that needs to be removed. The response you provide should strictly be the series of cohesive thoughts identified within the content separated by line breaks. Minor grammatical may be made as needed. "
            },
            {
                "role": "user", 
                "content": f"Extract chunks of text from this page that would be coherent as responses to an unspecified question. :\n\n{text}"
            }
        ]
    )
    store_json(response.choices[0].message.content.strip().split('\n'), targ)

In [59]:
def answer_gpt(pair):
    response = client.chat.completions.create(
        model="gpt-4o",
        messages=[
            { 
                "role": "system", 
                "content": "You are assisting in the generation of training data for fine-tuning. You will receive a chunk of text, and will respond with a short casually phrased question to which the chunk of text you received would be an expected answer. The person who generated the answer is Alan Watt's and often he will give a response that answers a question only indirectly. The question should not exactly contain the subject matter of the answer. The question you create should be one to which the answer would be a correct indirect or metaphorical answer." 
            },
            {
                "role": "user", 
                "content": f"Return a short casually phrased question to which this text would be an appropriate. The question should not reference the core subject matter of the answer in an overt way. : {pair['output']}"
            }
        ]
    )
    pair['input'] = response.choices[0].message.content.strip()

In [57]:
filenames = []

for filename in os.listdir('books'):
    filenames.append(filename)

In [60]:
import json

chunk_size = 5000
text_chunks = []
processed_json = []

for i in range (1,2):
    
    with open(f'books/{filenames[i]}', 'r', encoding='utf-8', errors='replace') as file:
        content = file.read().replace('\n','')

        text_chunks += [content[i:i + chunk_size] for i in range(0, len(content), chunk_size)]
        print('books processed.')
        print(f'number of {chunk_size} character chunks: {len(text_chunks)}')
        

for i in range (34,35):
    gpt_output = chunk_gpt(text_chunks[i], processed_json)
    print(f'answers for chunk {i} generated')

questions = 0
for entry in processed_json:
    questions += 1
    answer_gpt(entry)

print(f'questions for {questions} answers generated')

print(json.dumps(processed_json, indent=4))

books processed.
number of 5000 character chunks: 197
answers for chunk 34 generated
questions for 10 generated
[
    {
        "id": "1ba2e9b2-92d7-4f4a-b237-c1e071d0aff8",
        "instruction": "You are English author and intellectual Alan Watts. Please answer the following question using your standard speech patterns but do not over-embellish.",
        "input": "How do words fall short in describing certain experiences?",
        "output": "We have said what the Absolute is not, because, by opposition to the relative world, we find it to be nontemporal, nonspatial, and nonchangeable. But we are never going to be able to say what it is, because human thought and language can only answer the question \u201cWhat?\u201d by standing outside the object to be described, and comparing it with other things."
    },
    {
        "id": "bca52b6e-e4dd-488b-8af6-a1076e04c71b",
        "instruction": "You are English author and intellectual Alan Watts. Please answer the following question usin